#5.1 CNN 기초

[참고 블로그](https://untitledtblog.tistory.com/150)

##5.1.1 컴퓨터가 보는 이미지

- 일반적인 인공신경망은 다양한 형태의 입력에 대한 확장성이 떨어짐..
- 이미지에 대해서 자연스럽게 학습하기 위한 다른 방법이 필요함..

##5.1.2 컨볼루션


- **계층적으로 인식할 수 있도록 단계마다 이미지의 특징을 추출하는 방식을 사용**

- 각 단계에서는 이미지에 다양한 필터를 적용(**몇 개의 필터를 적용할지에 따라 채널이 결정됨..**)하여 각종 특징 추출..

##5.1.3 CNN 모델

기본 용어

- 필터(커널) : 도장..
- 채널 : 입력의 깊이..
- 패딩
- 스트라이드
- 특징 맵 : 필터를 적용한 결과
- 풀링 : 특성 맵의 크기를 줄이는 역할..

#5.2 CNN 모델 구현하기

- 일반적인 DNN 신경망은 그림을 벡터화해서 학습에 적용
- 이 벡터화 과정에서 인접 픽셀과의 상관관계가 의미 없어짐..(많은 정보의 손실)
- CNN은 이를 해결!

순서..

- 컨볼루션 -> 풀링 -> 컨볼루션 -> 드롭아웃 -> 풀링 -> 신경망 -> 드롭아웃 -> 신경망..
- CNN 신경망과 풀링으로 어느정도 이미지의 크기를 줄인 다음에 일반 신경망으로 학습

####기본 설정

In [1]:
#필수 도구들 임포트..

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

####데이터셋 불러오기

- 기본 전처리는 텐서화 및 정규화만 시행

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/train-labels-idx1-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/MNIST/raw



####뉴럴넷으로 Fashion MNIST 학습하기

- 원본 (28,28,1)
- 1차 conv (24, 24, 10) : kernel_size = (5,5), kernel_num = 10
- 1차 풀링 (12, 12, 10)
- 2차 conv (8, 8, 20) : kernel_size = (5,5), kernel_num = 20
- 2차 풀링 (4, 4, 20)

마지막의 shape이 (4,4,20)이므로 flatten()을 하면 4 * 4 * 20으로 320이다..

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #합성곱층 생성
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)        #필터 10개
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)       #필터 20개
        self.conv2_drop = nn.Dropout2d()        #드롭아웃 층
        self.fc1 = nn.Linear(320, 50)           #마지막 결과의 크기가 320..
        self.fc2 = nn.Linear(50, 10)            #출력 10개의 lable..

    #순전파 과정..
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)     #320인 이유가 중요!!
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

####하이퍼파라미터

모델과 최적화 객체 생성..

In [6]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

####학습하기

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()       #훈련모드
    for batch_idx, (data, target) in enumerate(train_loader):
        #데이터와 타깃 불러오고
        data, target = data.to(DEVICE), target.to(DEVICE)
        #역전파 알고리즘 조진다..
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        #200배치 마다 확인..
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

####테스트하기

In [8]:
def evaluate(model, test_loader):
    model.eval()        #평가 모드..
    test_loss = 0
    correct = 0

    #평가모드는 기울기 계산 불필요..
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

####코드 확인..

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.349893
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.272568
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.675466
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.630175
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.376897
[1] Test Loss: 0.1850, Accuracy: 94.51%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.542291
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.282521
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.418586
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.367931
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.214509
[2] Test Loss: 0.1193, Accuracy: 96.25%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.126010
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.215103
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.304305
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.380126
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.318418
[3] Test Loss: 0.0951, Accuracy: 97.06%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.289220
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.218907
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.345114


#5.3 ResNet으로 컬러 데이터셋에 적용하기

##5.3.1 ResNet 소개

- CNN의 응용
- 신경망을 깊게 쌓으면 오히려 성능이 나빠지는 문제를 해결하는 방법..
- **컨볼루션층의 출력에 전의 전 계층에 쓰였던 입력을 더함으로써 특징을 보존!!**

##5.3.2 CIFAR-10 데이터셋

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [11]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

하이퍼파라미터

In [12]:
EPOCHS     = 300
BATCH_SIZE = 128

데이터셋 불러오기

In [13]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),    #과적합방지 노이즈 추가
                       transforms.RandomHorizontalFlip(),       #과적합방지 노이즈 추가
                       transforms.ToTensor(),               
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data/cifar-10-python.tar.gz to ./.data


##5.3.3 CNN을 깊게 쌓는 방법

####ResNet 모델 만들기

기본이 되는 BasicBlock 클래스..

- conv층과 batchnormalize층이 2개씩 들어가있다..
- 전의 전 층의 입력을 그대로 더해줄 shortcut층도 있다..

In [14]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                #kernel_size=1은 그대로 나오는 것과 똑같다..
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    #데이터의 흐름은 아래와 같다..
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

위의 BasicBlock을 이용하여 만드는 ResNet 클래스..

In [15]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        #16채널에서 16채널을 내보내는 BasicBlock 2개
        #둘다 shortcut 없음
        self.layer1 = self._make_layer(16, 2, stride=1)

        #16채널을 받아 32채널을 출력하는 BasicBlock (shortcut 있음)
        #32채널을 받아 32채널을 출력하는 BasicBlock (shortcut 없음)
        self.layer2 = self._make_layer(32, 2, stride=2)

        #32채널을 받아 64채널을 출력하는 BasicBlock (shortcut 있음)
        #64채널을 받아 64채널을 출력하는 BasicBlock (shortcut 없음)
        self.layer3 = self._make_layer(64, 2, stride=2)

        #마지막 선형계산..
        #layer3의 결과인 (64,8,8)텐서를 평균풀링해서 원소의 갯수를 64개로 만든다
        self.linear = nn.Linear(64, num_classes)

    #층만들기..
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:  #len(strides)만큼의 BasicBlock을 만든다..
            layers.append(BasicBlock(self.in_planes, planes, stride))
            #이 줄때문에 첫 BasicBlock만 Shortcut을 갖도록 해준다.. (이 예제에서는!)
            self.in_planes = planes     
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)  #8,8로 평균풀링..
        out = out.view(out.size(0), -1) #flatten()
        out = self.linear(out)
        return out

####준비

학습효율을 높이기 위해 **학습률 감소**기법 사용!
- 최적화 함수의 학습률을 점점 낮춰서 정교하게 최적화한다..

In [16]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [17]:
#모델 출력..

print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

####학습하기

In [18]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

####테스트하기

In [19]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

####코드 돌려보기

시간관계로 5에포크만 반복..

In [21]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))
    if epoch >= 5:
        break

[1] Test Loss: 0.3950, Accuracy: 87.71%
[2] Test Loss: 0.3815, Accuracy: 88.06%
[3] Test Loss: 0.3812, Accuracy: 87.98%
[4] Test Loss: 0.3596, Accuracy: 88.84%
[5] Test Loss: 0.3779, Accuracy: 88.04%
